In [1]:
import tweepy
import os
import json
from IPython.display import JSON
import logging
import parser
import traceback
import sys
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import pandas as pd


In [2]:
def parser_twitter(): 
    engine = create_engine(os.environ['SHARED_DB_URI'])
    df = pd.read_sql('SELECT * FROM raws limit 1000', engine)
    df1 = (pd.concat({i: pd.io.json.json_normalize(x) for i, x in df.pop('raw_data').items()})
         .reset_index(level=1, drop=True)
         .join(df, rsuffix='raw',)
         .reset_index(drop=True))
    return df1
parser()

TypeError: 'module' object is not callable